In [286]:
import Impute
using Plots
using DataFrames
import CSV
using MLJ
using LossFunctions
import FileIO
using Missings
using StatsBase

# Load

In [315]:
features = DataFrame(CSV.File("features.csv",missingstring="missing"));
train = DataFrame(CSV.File("train.csv"));
test = DataFrame(CSV.File("test.csv"));
stores = DataFrame(CSV.File("stores.csv"));
features.StoreType = stores[features.Store,:Type];
features.Size = stores[features.Store,:Size];
features.Id1 = string.(features.Store,"_",features.Date);
select!(features, Not([:Store,:Date]));

In [316]:

train.Id1 = string.(train.Store,"_",train.Date);

test.Id1 = string.(test.Store,"_",test.Date);


In [317]:
train = innerjoin(features,train,on=:Id1,makeunique=true)
train.Id = string.(train.Store,"_",train.Dept,"_",train.Date);
select!(train, Not([:Id1,:IsHoliday_1]));
train;

In [318]:
test = innerjoin(features,test,on=:Id1,makeunique=true)
test.Id = string.(test.Store,"_",test.Dept,"_",test.Date);
select!(test, Not([:Id1,:IsHoliday_1]));
test;

In [319]:
train = coerce(train, :StoreType => Multiclass);
test = coerce(test, :StoreType => Multiclass);

In [320]:
schema(train)

┌──────────────┬──────────────────────────────────┬─────────────────────────────
│ _.names      │ _.types                          │ _.scitypes                 ⋯
├──────────────┼──────────────────────────────────┼─────────────────────────────
│ Temperature  │ Float64                          │ Continuous                 ⋯
│ Fuel_Price   │ Float64                          │ Continuous                 ⋯
│ MarkDown1    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown2    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown3    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown4    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown5    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ CPI          │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ Unemployment │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ IsHoliday    │ Bool       

# Describe

In [321]:
describe(allowmissing(train))

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Union
1,Temperature,60.0901,-2.06,62.09,100.14,0,"Union{Missing, Float64}"
2,Fuel_Price,3.36103,2.472,3.452,4.468,0,"Union{Missing, Float64}"
3,MarkDown1,7246.42,0.27,5347.45,88646.8,270889,"Union{Missing, Float64}"
4,MarkDown2,3334.63,-265.76,192.0,1.0452e5,310322,"Union{Missing, Float64}"
5,MarkDown3,1439.42,-29.1,24.6,1.41631e5,284479,"Union{Missing, Float64}"
6,MarkDown4,3383.17,0.22,1481.31,67474.9,286603,"Union{Missing, Float64}"
7,MarkDown5,4628.98,135.16,3359.45,1.08519e5,270138,"Union{Missing, Float64}"
8,CPI,171.202,126.064,182.319,227.233,0,"Union{Missing, Float64}"
9,Unemployment,7.96029,3.879,7.866,14.313,0,"Union{Missing, Float64}"


In [322]:
describe(allowmissing(test))

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Union
1,Temperature,53.9418,-7.29,54.47,101.95,0,"Union{Missing, Float64}"
2,Fuel_Price,3.58155,2.872,3.606,4.125,0,"Union{Missing, Float64}"
3,MarkDown1,7689.22,-2781.45,4842.29,103185.0,149,"Union{Missing, Float64}"
4,MarkDown2,3734.05,-35.74,742.59,71074.2,28627,"Union{Missing, Float64}"
5,MarkDown3,2403.09,-179.26,78.26,1.49483e5,9829,"Union{Missing, Float64}"
6,MarkDown4,3356.22,0.22,840.94,65344.6,12888,"Union{Missing, Float64}"
7,MarkDown5,3922.68,-185.17,2390.43,7.71448e5,0,"Union{Missing, Float64}"
8,CPI,176.961,131.236,192.304,228.976,38162,"Union{Missing, Float64}"
9,Unemployment,6.86873,3.684,6.806,10.199,38162,"Union{Missing, Float64}"


# Impute

In [323]:
a=hcat( allowmissing(train)[:,1:10],allowmissing(train)[:,12:14],allowmissing(train)[:,16] )
a=Matrix(a)
a=Impute.svd(a) # could try knn
train[:,3:7] = a[:,3:7];


In [324]:
a=hcat( allowmissing(test)[:,1:10],allowmissing(test)[:,12:14] )
a=Matrix(a)
a=Impute.svd(a)
test[:,3:6] = a[:,3:6];
test[:,8:9] = a[:,8:9];

In [325]:
# train = Impute.filter(train; dims=:rows) # drop missing rows
# test = Impute.filter(test; dims=:rows) # drop missing rows
# # train = Impute.substitute(train) # substitute (somehow!?) missing data

In [326]:
train = disallowmissing(train)
test = disallowmissing(test)

,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42
2,61.24,3.314,11421.3,3370.89,40.28,4646.79,6154.16
3,52.92,3.252,9696.28,292.1,103.78,1133.15,6612.69
4,56.23,3.211,883.59,4.17,74910.3,209.91,303.32
5,52.34,3.207,2460.03,4134.92,3838.35,150.57,6966.34
6,64.12,3.198,6343.16,3973.76,270.0,2928.9,10147.9
7,48.89,3.168,3504.83,4256.4,73.26,1636.8,2779.6
8,56.02,3.098,8231.71,4364.88,274.0,358.15,2834.02
9,44.79,3.108,12659.5,37101.1,174.78,74.46,1208.86


# Unpack

In [327]:
# y, X = unpack(train, ==(:Weekly_Sales), colname -> true) # 
y, x_train = unpack(train, 
    ==(:Weekly_Sales), 
    colname -> colname != :Date &&  colname !=:Id
#     !=([:Date, :Id])
#     !=(:Date)
    );
x_test = select(test, Not([:Date, :Id]));

# One Hot and Standardize

This is a mess

In [328]:



mach = fit!(machine(OneHotEncoder(), x_train))

x_train = MLJ.transform(mach, x_train)

x_test = MLJ.transform(mach, x_test)

x_train[!,:StoreType__A] = convert.(Bool,x_train[!,:StoreType__A])
x_train[!,:StoreType__B] = convert.(Bool,x_train[!,:StoreType__B])
x_train[!,:StoreType__C] = convert.(Bool,x_train[!,:StoreType__C])
x_train[!,:Size] = convert.(Float64,x_train[!,:Size])

x_test[!,:StoreType__A] = convert.(Bool,x_test[!,:StoreType__A])
x_test[!,:StoreType__B] = convert.(Bool,x_test[!,:StoreType__B])
x_test[!,:StoreType__C] = convert.(Bool,x_test[!,:StoreType__C])
test[!,:Size] = convert.(Float64,x_test[!,:Size])

for i in 1:length(x_train[1,:])
    if typeof(x_train[1,i]) == Float64
        dt = StatsBase.fit(UnitRangeTransform, x_train[:,i])
        x_train[:,i] = StatsBase.transform(dt, x_train[:,i])
        x_test[:,i] = StatsBase.transform(dt, x_test[:,i])
    end
end

┌ Info: Training Machine{OneHotEncoder,…} @828.
└ @ MLJBase /Users/alex/.julia/packages/MLJBase/hLtde/src/machines.jl:342
┌ Info: Spawning 3 sub-features to one-hot encode feature :StoreType.
└ @ MLJModels /Users/alex/.julia/packages/MLJModels/E8BbE/src/builtins/Transformers.jl:1142


LoadError: InexactError: Int64(0.6302673385765398)

In [329]:
describe(x_train)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,Temperature,0.608122,0.0,0.627691,1.0,0,Float64
2,Fuel_Price,0.445404,0.0,0.490982,1.0,0,Float64
3,MarkDown1,0.0745849,0.0,0.0658706,1.0,0,Float64
4,MarkDown2,0.146615,0.0,0.142355,1.0,0,Float64
5,MarkDown3,0.0690661,0.0,0.0665737,1.0,0,Float64
6,MarkDown4,0.0752952,0.0,0.0690309,1.0,0,Float64
7,MarkDown5,0.0396928,0.0,0.0353899,1.0,0,Float64
8,CPI,0.446165,0.0,0.556049,1.0,0,Float64
9,Unemployment,0.391153,0.0,0.382116,1.0,0,Float64


In [330]:
describe(x_test)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,Temperature,0.547963,-0.0511742,0.553131,1.01771,0,Float64
2,Fuel_Price,0.555885,0.200401,0.568136,0.828156,0,Float64
3,MarkDown1,0.0866604,-0.0313799,0.0544686,1.164,0,Float64
4,MarkDown2,0.154671,0.0941569,0.134328,0.720066,0,Float64
5,MarkDown3,0.0758426,0.0510615,0.0609155,1.0521,0,Float64
6,MarkDown4,0.0799343,-0.00125495,0.0464806,0.969618,0,Float64
7,MarkDown5,0.0349454,-0.00295551,0.0208081,7.11648,0,Float64
8,CPI,0.448187,-0.665336,0.378272,1.20132,0,Float64
9,Unemployment,0.285105,-0.0591712,0.279088,0.605712,0,Float64


# Save

In [331]:
FileIO.save("preprocessed.jld2", Dict("x_train" => x_train, "x_test" => x_test, "y" => y) )